---
title: "5: Seeing (data) is believing"
sidebar: modules
---

- [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg){width=30%}](https://colab.research.google.com/github/nils-holmberg/cda-gai/blob/main/jnb/apws-module-5.ipynb){target="_blank"}

In [ ]:
# Empty code cell with Python comment preceded by hash sign "# "

### Generate a simple Python program

**Planned focus:** Draw a circle with NumPy and save PNG

- **Key topics / activities:** NumPy arrays as images, basic image I/O
- **Example / mini-project:** Create 100×100 image: green background with white circle; save as PNG

In [ ]:
# TODO: Implement: Generate a simple Python program for Day 5
# Suggested starting hints from schedule:
# Title: Draw a circle with NumPy and save PNG
# Key topics / activities: NumPy arrays as images, basic image I/O
# Example / Mini-project: Create 100×100 image: green background with white circle; save as PNG

# Write your code here...

### Solve problem in your own work

**Planned focus:** Simulate an interaction & visualize

- **Key topics / activities:** Bivariate median splits, data simulation, error bars
- **Example / mini-project:** Generate data showing A×B interaction and plot grouped bars with error bars

In [ ]:
# TODO: Implement: Solve problem in your own work for Day 5
# Suggested starting hints from schedule:
# Title: Simulate an interaction & visualize
# Key topics / activities: Bivariate median splits, data simulation, error bars
# Example / Mini-project: Generate data showing A×B interaction and plot grouped bars with error bars

# Write your code here...

### Perform data science using GenAI

**Planned focus:** Median split & cleanup on penguins (viz-ready)

- **Key topics / activities:** Recode by median, drop NAs, prep for plotting
- **Example / mini-project:** Recreate median-split categorical variable and export a clean, plot-ready table

In [ ]:
# TODO: Implement: Perform data science using GenAI for Day 5
# Suggested starting hints from schedule:
# Title: Median split & cleanup on penguins (viz-ready)
# Key topics / activities: Recode by median, drop NAs, prep for plotting
# Example / Mini-project: Recreate median-split categorical variable and export a clean, plot-ready table

# Write your code here...